In [3]:
#!pip install transformers

In [1]:
#!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
# https://huggingface.co/jason9693/koelectra-small-v3-generator-apeach

In [37]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("beomi/korean-hatespeech-multilabel")
model = AutoModelForSequenceClassification.from_pretrained("beomi/korean-hatespeech-multilabel")

C:\Users\b1005\anaconda3\envs\langchain\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\b1005\.cache\huggingface\hub\models--jason9693--koelectra-small-v3-generator-apeach. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [38]:
import pandas as pd
df = pd.read_csv("APEACH_데이터_라벨링완료.csv", encoding="utf-8-sig")
df

,text,is_immoral,label_words,label_numbers
0,내년에 네이버 게임회사 같은데서 사람 엄청 뽑드만. 이럴 줄 알았음 공대를 갈것을 ...,True,['Education'],[0]
1,여자가 아이를 돌보고 남자가 밖에서 돈을 벌어온다' 다 옛날 말이죠. 요즘은 다 반...,True,['Gender sterotypes'],[4]
2,"오베라는 남자'라는 책에서 부인의 신체 변화에 따라 남편인 주인공, 오베가 차를 바...",True,['Disabled'],[6]
3,10년동안 일한 보람이 드디어 생기나보네,True,['Age and social status'],[7]
4,10대들은 모르는 추억의 노래,True,['Age and social status'],[7]
...,...,...,...,...
3765,일본놈들이 하는 짓이 다 그렇지 뭐,False,['Racism'],[3]
3766,오잉? 저게 이쁘다는거야? 장님이야뭐야 눈이 어떻게 된거야? 없는거야? 눈어디갔니?...,False,['Disabled'],[6]
3767,아니 왜 이렇게 태웠어? 팔다리만 길어가지고 배만 툭튀어나온게 까맣게 태우니까 흑인...,False,['Racism'],[3]
3768,"한녀들, 김치녀들 취집가려고 한다.",False,['Gender sterotypes'],[4]


In [39]:
sampled_data = df.groupby("label_numbers").first().reset_index()
sampled_data = sampled_data.drop(columns = ['is_immoral','label_words'])
sampled_data

,label_numbers,text
0,[0],내년에 네이버 게임회사 같은데서 사람 엄청 뽑드만. 이럴 줄 알았음 공대를 갈것을 ...
1,[3],가끔 스페인처럼 낮잠 문화(시에스타)가 부럽다는 생각이 들어. 잠깐의 휴식이 있으면...
2,[4],여자가 아이를 돌보고 남자가 밖에서 돈을 벌어온다' 다 옛날 말이죠. 요즘은 다 반...
3,[6],"오베라는 남자'라는 책에서 부인의 신체 변화에 따라 남편인 주인공, 오베가 차를 바..."
4,[7],10년동안 일한 보람이 드디어 생기나보네


In [40]:
samples = [{"text": text, "label": label[1]} for text, label in zip(sampled_data['text'], sampled_data['label_numbers'])]
samples

[{'text': '내년에 네이버 게임회사 같은데서 사람 엄청 뽑드만. 이럴 줄 알았음 공대를 갈것을 문과생은 웁니다ㅠㅠ',
  'label': '0'},
 {'text': '가끔 스페인처럼 낮잠 문화(시에스타)가 부럽다는 생각이 들어. 잠깐의 휴식이 있으면 다시 일할 기운을 얻을 수 있잖아!',
  'label': '3'},
 {'text': "여자가 아이를 돌보고 남자가 밖에서 돈을 벌어온다' 다 옛날 말이죠. 요즘은 다 반반씩!!", 'label': '4'},
 {'text': "오베라는 남자'라는 책에서 부인의 신체 변화에 따라 남편인 주인공, 오베가 차를 바꾸는 부분이 인상적이였어요. 아내가 휠체어를 타게되서 휠체어까지 탑승할 수있는 차를 바꾸고 노력하는 부분이 세밀하게 표현된것이 아니라 그저 바뀐 자동차 모델로만 나열되는데 인상적이였습니다.",
  'label': '6'},
 {'text': '10년동안 일한 보람이 드디어 생기나보네', 'label': '7'}]

In [41]:
text = df['text']
label = df['label_numbers']

In [42]:
test = [{"text": text, "label": label[1]} for text, label in zip(df['text'], df['label_numbers'])]
test

[{'text': '내년에 네이버 게임회사 같은데서 사람 엄청 뽑드만. 이럴 줄 알았음 공대를 갈것을 문과생은 웁니다ㅠㅠ',
  'label': '0'},
 {'text': "여자가 아이를 돌보고 남자가 밖에서 돈을 벌어온다' 다 옛날 말이죠. 요즘은 다 반반씩!!", 'label': '4'},
 {'text': "오베라는 남자'라는 책에서 부인의 신체 변화에 따라 남편인 주인공, 오베가 차를 바꾸는 부분이 인상적이였어요. 아내가 휠체어를 타게되서 휠체어까지 탑승할 수있는 차를 바꾸고 노력하는 부분이 세밀하게 표현된것이 아니라 그저 바뀐 자동차 모델로만 나열되는데 인상적이였습니다.",
  'label': '6'},
 {'text': '10년동안 일한 보람이 드디어 생기나보네', 'label': '7'},
 {'text': '10대들은 모르는 추억의 노래', 'label': '7'},
 {'text': '1개월차가 의견 내는 건 안받아 주나봐.', 'label': '7'},
 {'text': '20대에 취업을 한 친구가 있어.', 'label': '7'},
 {'text': '30살도 새로시작하기에 늦지 않아요', 'label': '7'},
 {'text': '3기 신도시 덕분에? 경기도민이 됬어요. 좋네요.', 'label': '0'},
 {'text': '3살한테도 배울점은 있다', 'label': '7'},
 {'text': '90년대 남자가수 중에 치마를 무대의상으로 입고 나와서 방송불가 판정 받은 사람이 있다더라? 요즘에는 있을 수 없는 일이지.',
  'label': '4'},
 {'text': '90년대생은 퇴사도 쉽게 한다던데 끈기가 없어서 그런가.', 'label': '7'},
 {'text': '가끔 스페인처럼 낮잠 문화(시에스타)가 부럽다는 생각이 들어. 잠깐의 휴식이 있으면 다시 일할 기운을 얻을 수 있잖아!',
  'label': '3'},
 {'text': '가슴에 종양생겨서 수술 두번 했다는 친구가 회사는 계속 다닌다는게 이해가 안되네

In [ ]:
pipe = pipeline(
    task="text-classification",
    model=model,
    tokenizer=tokenizer,
    device=0  # GPU 사용, CPU 사용 시 -1로 변경
)

restored_reviews = []

# 테스트 데이터 순회
for data in test:  # `test`는 리스트 또는 시리즈 형태여야 합니다.
    query = data.get('text', '')
    outputs = pipe(query)
    results = outputs[0]['label']
    print(outputs)
    restored_reviews.append(results)

Device set to use cuda:0


[{'label': 'Spoiled', 'score': 0.5272491574287415}]
[{'label': 'Default', 'score': 0.5716819167137146}]
[{'label': 'Default', 'score': 0.8887683153152466}]
[{'label': 'Default', 'score': 0.9075252413749695}]
[{'label': 'Default', 'score': 0.5094397664070129}]
[{'label': 'Default', 'score': 0.7147237062454224}]
[{'label': 'Default', 'score': 0.502525269985199}]
[{'label': 'Default', 'score': 0.854491651058197}]
[{'label': 'Default', 'score': 0.8986104130744934}]
[{'label': 'Default', 'score': 0.5783632397651672}]
[{'label': 'Spoiled', 'score': 0.6252617835998535}]
[{'label': 'Spoiled', 'score': 0.5027925968170166}]
[{'label': 'Default', 'score': 0.8156622052192688}]
[{'label': 'Default', 'score': 0.6502314209938049}]
[{'label': 'Default', 'score': 0.9006496071815491}]
[{'label': 'Default', 'score': 0.8574675917625427}]
[{'label': 'Default', 'score': 0.8527942895889282}]
[{'label': 'Default', 'score': 0.9129802584648132}]
[{'label': 'Default', 'score': 0.9188516139984131}]
[{'label': 'De

In [ ]:
restored_reviews